In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import sklearn
import sys
IS_KAGGLE="kaggle_secrets" in sys.modules

np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1,as_frame=False)
mnist.keys()

/opt/conda/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [4]:
X,y=mnist['data'],mnist['target']
y=y.astype(np.uint8)
X_train,y_train=X[:60000],y[:60000]
X_test,y_test=X[60000:],y[60000:]

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

In [6]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': [3,4,5], 'weights': ['uniform','distance']}
knn_clf = KNeighborsClassifier()
grid_search = GridSearchCV(knn_clf,param_grid,cv=5,return_train_score=True,verbose=3)
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END n_neighbors=3, weights=uniform;, score=(train=0.985, test=0.972) total time=  15.9s
[CV 2/5] END n_neighbors=3, weights=uniform;, score=(train=0.986, test=0.971) total time=  16.3s
[CV 3/5] END n_neighbors=3, weights=uniform;, score=(train=0.986, test=0.969) total time=  15.8s
[CV 4/5] END n_neighbors=3, weights=uniform;, score=(train=0.986, test=0.969) total time=  16.3s
[CV 5/5] END n_neighbors=3, weights=uniform;, score=(train=0.986, test=0.970) total time=  15.8s
[CV 1/5] END n_neighbors=3, weights=distance;, score=(train=1.000, test=0.972) total time=  16.0s
[CV 2/5] END n_neighbors=3, weights=distance;, score=(train=1.000, test=0.972) total time=  15.4s
[CV 3/5] END n_neighbors=3, weights=distance;, score=(train=1.000, test=0.970) total time=  15.3s
[CV 4/5] END n_neighbors=3, weights=distance;, score=(train=1.000, test=0.970) total time=  15.9s
[CV 5/5] END n_neighbors=3, weights=distance;, score=(train=1.0

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 4, 5],
                         'weights': ['uniform', 'distance']},
             return_train_score=True, verbose=3)

In [7]:
grid_search.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [8]:
grid_search.best_score_

0.9716166666666666

In [9]:
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.9714